In [1]:
from google.colab import drive
drive.mount("/content/gdrive",force_remount=True)

Mounted at /content/gdrive


In [2]:
import pickle
import numpy as np
from numpy import mean
from numpy import std
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from keras.backend import clear_session

In [11]:
path_data = "/content/gdrive/MyDrive/BK-DATN/data_final_23616.pkl"
load_paths = ["/content/gdrive/MyDrive/BK-DATN/save_model/param_6080138_bz_64_loss_0.00802_acc_0.7498.hdf5", 
              "/content/gdrive/MyDrive/BK-DATN/save_model/param_6080138_bz_64_loss_0.01189_acc_0.7389.hdf5",
              "/content/gdrive/MyDrive/BK-DATN/save_model/param_6080138_bz_64_loss_0.01567_acc_0.7331.hdf5",
              "/content/gdrive/MyDrive/BK-DATN/save_model/param_6080138_bz_64_loss_0.02000_acc_0.7089.hdf5"]
load_paths = ["/content/gdrive/MyDrive/BK-DATN/save_model/param_6080138_bz_64_loss_0.01499_acc_0.6837.hdf5", 
              "/content/gdrive/MyDrive/BK-DATN/save_model/param_6080138_bz_64_loss_0.01812_acc_0.6509.hdf5",
              "/content/gdrive/MyDrive/BK-DATN/save_model/param_6080138_bz_64_loss_0.02423_acc_0.7811.hdf5"]

In [12]:

with open(path_data, 'rb') as f:
    list_data = pickle.load(f)

In [13]:
print(len(list_data))
print(list_data[3])
MIST_bag = 23616

21192
['50042c8722f85fa1b27d6c8d1def0b78708fd1b0c8a3b82996551df0d22cc492', [[35, 3], [62, 1], [261, 1], [282, 1], [371, 1], [399, 2], [450, 1], [541, 1], [698, 1], [711, 1], [848, 1], [865, 3], [1012, 2], [1023, 2], [1198, 8], [1205, 1], [1354, 1], [1370, 1], [1399, 12], [1432, 1], [1543, 1], [1598, 1], [1599, 2], [1656, 1], [1731, 1], [1782, 1], [2003, 4], [2016, 1], [2030, 23], [2058, 1], [2066, 2], [2087, 5], [2092, 114], [2225, 3], [2252, 1], [2507, 1], [2552, 2], [2700, 1], [2753, 1], [2764, 5], [2790, 1], [2864, 1], [2952, 96], [3173, 1], [3307, 76], [3559, 2], [3563, 2], [3566, 1], [3664, 1], [3693, 1], [3739, 69], [3927, 1], [4046, 23], [4097, 1], [4187, 1], [4227, 2], [4239, 2], [4324, 2], [4331, 3], [4463, 10], [4548, 1], [4595, 65], [4603, 1], [4616, 19], [4624, 9], [4776, 1], [4832, 52], [4940, 1], [5146, 1], [5167, 2], [5228, 14], [5500, 14], [5798, 1], [5828, 40], [5854, 2], [5915, 5], [5974, 3], [6095, 2], [6151, 4], [6330, 2], [6342, 1], [6400, 1], [6544, 2], [6595, 2],

In [14]:
# get data
X = []
y = []
#choose_tag_index = [29, 47, 59, 67, 68, 71, 88, 101, 105, 115, 122, 123, 124, 125, 126, 147, 151] [59, 68, 88, 101, 105, 115, 122, 125, 126, 147, 151]
choose_tag_index =  [59, 68, 88, 101, 105, 115, 122, 125, 147, 151] # [68, 88, 101, 105, 125, 126, 147, 151] # [59, 68, 88, 101, 105, 122, 125, 126, 147, 151]
def unzip_mist(data,MIST_bag ):
    temp_value = [0]*MIST_bag
    for value in data:
        temp_value[value[0]] = value[1]
    return temp_value
for i,data in enumerate(list_data[15000:]):
    #print(i)
    X.append(unzip_mist(data[1], MIST_bag))
    tag = []
    for tag_index in choose_tag_index:
        tag.append(data[2][tag_index])
    y.append(tag)
TestX, TestY = np.array(X.copy()), np.array(y.copy())
list_data.clear()
X.clear()
y.clear()

model = Sequential()
model.add(Dense(256, input_dim=MIST_bag, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='sigmoid'))


**Tập train**

In [ ]:
for load_path in load_paths:
    print("=======================================================================")
    print(f"model : {load_path.split('/')[-1]}")
    model.load_weights(load_path)
    TP = 0
    FP = 0
    FN = 0
    for i in range(0,len(TestY)):
        #print(i)
        Predict_Y = model.predict(TestX[i:i+1]).round()
        for j in range(0,10):
            if Predict_Y[0][j] == 1 and TestY[i][j] == 1:
                TP+=1
            if Predict_Y[0][j] == 0 and TestY[i][j] == 1:
                FP+=1
            if Predict_Y[0][j] == 1 and TestY[i][j] == 0:
                FN+=1

    micro_average_precision = (TP)/(TP+FP)
    print(f"micro_average_precision = {micro_average_precision}")
    micro_average_recall = (TP)/(TP+FN)
    print(f"micro_average_recall = {micro_average_recall}")
    F_score = 5/(1/micro_average_precision + 4/micro_average_recall)
    print(f"F_score = {F_score}")

model : param_6080138_bz_64_loss_0.00802_acc_0.7498.hdf5
micro_average_precision = 0.9936823493897969
micro_average_recall = 0.9599641718915991
F_score = 0.9665234935362365
model : param_6080138_bz_64_loss_0.01189_acc_0.7389.hdf5
micro_average_precision = 0.9911596015352107
micro_average_recall = 0.9610905517573021
F_score = 0.9669575168073237
model : param_6080138_bz_64_loss_0.01567_acc_0.7331.hdf5
micro_average_precision = 0.9904264953210574
micro_average_recall = 0.962684299322009
F_score = 0.9681077070933586
model : param_6080138_bz_64_loss_0.02000_acc_0.7089.hdf5
micro_average_precision = 0.9787830436845056
micro_average_recall = 0.9646598805703721
F_score = 0.9674518128266114


**Tập Test**

In [ ]:
for load_path in load_paths:
    print("=======================================================================")
    print(f"model : {load_path.split('/')[-1]}")
    model.load_weights(load_path)
    TP = 0
    FP = 0
    FN = 0
    for i in range(0,len(TestY)):
        #print(i)
        Predict_Y = model.predict(TestX[i:i+1]).round()
        for j in range(0,10):
            if Predict_Y[0][j] == 1 and TestY[i][j] == 1:
                TP+=1
            if Predict_Y[0][j] == 0 and TestY[i][j] == 1:
                FP+=1
            if Predict_Y[0][j] == 1 and TestY[i][j] == 0:
                FN+=1

    micro_average_precision = (TP)/(TP+FP)
    print(f"micro_average_precision = {micro_average_precision}")
    micro_average_recall = (TP)/(TP+FN)a
    print(f"micro_average_recall = {micro_average_recall}")
    F_score = 5/(1/micro_average_precision + 4/micro_average_recall)
    print(f"F_score = {F_score}")

model : param_6080138_bz_64_loss_0.00802_acc_0.7498.hdf5
micro_average_precision = 0.9536432905963408
micro_average_recall = 0.7529449908137901
F_score = 0.7860296495754138
model : param_6080138_bz_64_loss_0.01189_acc_0.7389.hdf5
micro_average_precision = 0.9454761144317196
micro_average_recall = 0.7608033190145758
F_score = 0.7917319374928361
model : param_6080138_bz_64_loss_0.01567_acc_0.7331.hdf5
micro_average_precision = 0.9379020851393896
micro_average_recall = 0.7563192170425697
F_score = 0.786784350049375
model : param_6080138_bz_64_loss_0.02000_acc_0.7089.hdf5
micro_average_precision = 0.9375370716795182
micro_average_recall = 0.7778324563727903
F_score = 0.805267076850727


In [16]:
for load_path in load_paths:
    print("=======================================================================")
    print(f"model : {load_path.split('/')[-1]}")
    model.load_weights(load_path)
    TP = 0
    FP = 0
    FN = 0
    for i in range(0,len(TestY)):
        #print(i)
        Predict_Y = model.predict(TestX[i:i+1]).round()
        for j in range(0,10):
            if Predict_Y[0][j] == 1 and TestY[i][j] == 1:
                TP+=1
            if Predict_Y[0][j] == 0 and TestY[i][j] == 1:
                FP+=1
            if Predict_Y[0][j] == 1 and TestY[i][j] == 0:
                FN+=1

    micro_average_precision = (TP)/(TP+FP)
    print(f"micro_average_precision = {micro_average_precision}")
    micro_average_recall = (TP)/(TP+FN)
    print(f"micro_average_recall = {micro_average_recall}")
    F_score = 5/(1/micro_average_precision + 4/micro_average_recall)
    print(f"F_score = {F_score}")

model : param_6080138_bz_64_loss_0.01499_acc_0.6837.hdf5
micro_average_precision = 0.935301093688233
micro_average_recall = 0.7768130745658836
F_score = 0.8040629838009289
model : param_6080138_bz_64_loss_0.01812_acc_0.6509.hdf5
micro_average_precision = 0.9501471427943954
micro_average_recall = 0.7566546573857078
F_score = 0.7887808755642743
model : param_6080138_bz_64_loss_0.02423_acc_0.7811.hdf5
micro_average_precision = 0.9468529011288268
micro_average_recall = 0.7537412587412587
F_score = 0.7857939591884345


**UNMISMAS**

In [6]:
path_data = "/content/gdrive/MyDrive/BK-DATN/data_final_23616_UNMISMAC.pkl"
load_paths = ["/content/gdrive/MyDrive/BK-DATN/save_model/param_6080138_UNMISMAC_bz_64_loss_0.00292_acc_0.6946.hdf5", 
              "/content/gdrive/MyDrive/BK-DATN/save_model/param_6080138_UNMISMAC_bz_64_loss_0.00497_acc_0.6820.hdf5",
              "/content/gdrive/MyDrive/BK-DATN/save_model/param_6080138_UNMISMAC_bz_64_loss_0.00930_acc_0.7252.hdf5",
              "/content/gdrive/MyDrive/BK-DATN/save_model/param_6080138_UNMISMAC_bz_64_loss_0.00989_acc_0.6665.hdf5"]

In [7]:

with open(path_data, 'rb') as f:
    list_data = pickle.load(f)

In [8]:
print(len(list_data))
print(list_data[3])
MIST_bag = 23616

21192
['50042c8722f85fa1b27d6c8d1def0b78708fd1b0c8a3b82996551df0d22cc492', [[104, 2], [117, 11], [227, 1], [243, 1], [493, 1], [527, 1], [530, 12], [636, 1], [647, 2], [673, 1], [727, 2], [741, 3], [781, 36], [929, 10], [998, 1], [1095, 2], [1121, 14], [1126, 1], [1159, 1], [1162, 2], [1188, 1], [1240, 3], [1478, 2], [1621, 1], [1630, 4], [1644, 5], [1658, 3], [1772, 48], [1866, 168], [1890, 38], [1915, 1], [1978, 18], [2154, 2], [2174, 1], [2232, 1], [2284, 1], [2299, 23], [2396, 1], [2432, 3], [2546, 2], [2617, 65], [2648, 1], [2666, 1], [2761, 2], [2879, 3], [2901, 2], [3074, 1], [3144, 2], [3175, 2], [3192, 1], [3276, 3], [3316, 2], [3352, 2], [3506, 1], [3509, 305], [3519, 3], [3682, 2], [3825, 1], [3921, 1], [4014, 1], [4104, 1], [4175, 1], [4183, 2], [4416, 1], [4548, 10], [4580, 5], [4602, 3], [4667, 1], [4705, 12], [4773, 1], [4788, 141], [4814, 1], [4866, 14], [5025, 2], [5071, 2], [5077, 4], [5279, 2], [5298, 5], [5312, 3], [5339, 2], [5371, 1], [5424, 1], [5431, 1], [5468, 

In [9]:
# get data
X = []
y = []
#choose_tag_index = [29, 47, 59, 67, 68, 71, 88, 101, 105, 115, 122, 123, 124, 125, 126, 147, 151] [59, 68, 88, 101, 105, 115, 122, 125, 126, 147, 151]
choose_tag_index =  [59, 68, 88, 101, 105, 115, 122, 125, 147, 151] # [68, 88, 101, 105, 125, 126, 147, 151] # [59, 68, 88, 101, 105, 122, 125, 126, 147, 151]
def unzip_mist(data,MIST_bag ):
    temp_value = [0]*MIST_bag
    for value in data:
        temp_value[value[0]] = value[1]
    return temp_value
for i,data in enumerate(list_data[15000:]):
    #print(i)
    X.append(unzip_mist(data[1], MIST_bag))
    tag = []
    for tag_index in choose_tag_index:
        tag.append(data[2][tag_index])
    y.append(tag)
TestX, TestY = np.array(X.copy()), np.array(y.copy())
list_data.clear()
X.clear()
y.clear()

model = Sequential()
model.add(Dense(256, input_dim=MIST_bag, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='sigmoid'))


In [10]:
for load_path in load_paths:
    print("=======================================================================")
    print(f"model : {load_path.split('/')[-1]}")
    model.load_weights(load_path)
    TP = 0
    FP = 0
    FN = 0
    for i in range(0,len(TestY)):
        #print(i)
        Predict_Y = model.predict(TestX[i:i+1]).round()
        for j in range(0,10):
            if Predict_Y[0][j] == 1 and TestY[i][j] == 1:
                TP+=1
            if Predict_Y[0][j] == 0 and TestY[i][j] == 1:
                FP+=1
            if Predict_Y[0][j] == 1 and TestY[i][j] == 0:
                FN+=1

    micro_average_precision = (TP)/(TP+FP)
    print(f"micro_average_precision = {micro_average_precision}")
    micro_average_recall = (TP)/(TP+FN)
    print(f"micro_average_recall = {micro_average_recall}")
    F_score = 5/(1/micro_average_precision + 4/micro_average_recall)
    print(f"F_score = {F_score}")

model : param_6080138_UNMISMAC_bz_64_loss_0.00292_acc_0.6946.hdf5
micro_average_precision = 0.9449202793516932
micro_average_recall = 0.7717114467123435
F_score = 0.8010798733941538
model : param_6080138_UNMISMAC_bz_64_loss_0.00497_acc_0.6820.hdf5
micro_average_precision = 0.9502789124610181
micro_average_recall = 0.7620909507203494
F_score = 0.7935198022343991
model : param_6080138_UNMISMAC_bz_64_loss_0.00930_acc_0.7252.hdf5
micro_average_precision = 0.9496200641279045
micro_average_recall = 0.7657976763955795
F_score = 0.7966395224584546
model : param_6080138_UNMISMAC_bz_64_loss_0.00989_acc_0.6665.hdf5
micro_average_precision = 0.9506302982386788
micro_average_recall = 0.7591904026939806
F_score = 0.7910511041747381
